In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from contextlib import ExitStack
from tensorflow import keras
from tensorflow.train import BytesList, FloatList, Int64List
from tensorflow.train import Features, Feature, Example
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.datasets import fashion_mnist

In [2]:
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()
X_train, y_train = X_train_full[5000:], y_train_full[5000:]
X_val, y_val = X_train_full[:5000], y_train_full[:5000]

In [3]:
keras.backend.clear_session()

In [4]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(buffer_size=len(X_train))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [5]:
def create_dir(dir_prefix):
    """
    """
    dir_path = os.path.join("datasets", dir_prefix)
    os.makedirs(dir_path, exist_ok=True)
    return dir_path

In [6]:
def create_proto_example(image, label):
    """
    """
    image = tf.io.serialize_tensor(image)
    return Example(features=Features(feature={
        "image": Feature(bytes_list=BytesList(value=[image.numpy()])),
        "label": Feature(int64_list=Int64List(value=[label]))
    }))

In [7]:
def write_tfrecords(name, dataset, n_shards=10):
    """
    """
    paths = ["{}-{:05d}-of-{:05d}.tfrecord".format(name, index, n_shards)
             for index in range(1, n_shards + 1)]
    with ExitStack() as stack:
        writers = [stack.enter_context(tf.io.TFRecordWriter(path))
                   for path in paths]
        for index, (image, label) in dataset.enumerate():
            shard = index % n_shards
            example = create_proto_example(image, label)
            writers[shard].write(example.SerializeToString())
    return paths

In [8]:
dir_path = create_dir("fashion_mnist")
train_filepaths = write_tfrecords(os.path.join(dir_path,"train_dataset"), train_dataset)
valid_filepaths = write_tfrecords(os.path.join(dir_path,"val_dataset"), val_dataset)
test_filepaths = write_tfrecords(os.path.join(dir_path,"test_dataset"), test_dataset)

In [9]:
def preprocess(tfrecord):
    features_description = {
        "image": tf.io.FixedLenFeature([], tf.string, default_value=""),
        "label": tf.io.FixedLenFeature([], tf.int64, default_value=-1)
    }
    
    example = tf.io.parse_single_example(tfrecord, features_description)
    image = tf.io.parse_tensor(example["image"], out_type=tf.uint8)
    image = tf.reshape(image, [28,28])
    return image, example["label"]

def mnist_dataset(filepaths, buffer_size=None, n_threads=5, batch_size=32):
    dataset = tf.data.TFRecordDataset(filepaths)
    dataset = dataset.map(preprocess, num_parallel_calls=n_threads)
    if buffer_size:
        dataset = dataset.shuffle(buffer_size=buffer_size)
    dataset = dataset.batch(batch_size=batch_size)
    return dataset.prefetch(1)

In [10]:
train_dataset = mnist_dataset(train_filepaths, buffer_size=len(X_train))
test_dataset = mnist_dataset(test_filepaths)
val_dataset = mnist_dataset(valid_filepaths)

In [11]:
# Strip the label from the dataset
sample_dataset_batches = train_dataset.take(100).map(lambda image, label: image)
sample_dataset = np.concatenate(list(sample_dataset_batches), axis=0).astype(np.float32)

standardization = keras.layers.Normalization(input_shape=[28,28])
standardization.adapt(sample_dataset)

model = keras.models.Sequential()
model.add(standardization)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(10, activation="softmax"))

In [12]:
model.compile(loss="sparse_categorical_crossentropy", 
              optimizer="nadam", metrics=["accuracy"])

In [13]:
from datetime import datetime


In [14]:
log_dir = os.path.join("my_logs", 
                       "run_" + datetime.now().strftime("%Y%m%d_%H%M%S"))

tensorboard_cb = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir, histogram_freq=1, profile_batch=10)

model.fit(train_dataset, epochs=5, validation_data=val_dataset, callbacks=[tensorboard_cb])

Epoch 1/5
1719/1719 [==============================] - 5s 2ms/step - loss: 0.4451 - accuracy: 0.8409 - val_loss: 0.3615 - val_accuracy: 0.8740
Epoch 2/5
1719/1719 [==============================] - 4s 2ms/step - loss: 0.3333 - accuracy: 0.8783 - val_loss: 0.3413 - val_accuracy: 0.8774
Epoch 3/5
1719/1719 [==============================] - 4s 2ms/step - loss: 0.2985 - accuracy: 0.8899 - val_loss: 0.3302 - val_accuracy: 0.8830
Epoch 4/5
1719/1719 [==============================] - 4s 2ms/step - loss: 0.2744 - accuracy: 0.8985 - val_loss: 0.3279 - val_accuracy: 0.8896
Epoch 5/5
1719/1719 [==============================] - 4s 2ms/step - loss: 0.2530 - accuracy: 0.9067 - val_loss: 0.3261 - val_accuracy: 0.8864
